# 文本分析基础
## 高攀

In [1]:
import numpy as np
import pandas as pd
import re
from string import punctuation
import codecs
from pandas.core.frame import DataFrame
import jieba.posseg as pseg
from gensim import corpora, models, similarities
import logging
from gensim.models.word2vec import LineSentence, Word2Vec

In [2]:
cd C:\Users\asus\Desktop\数据中心\2019文本分析\电脑商品评论文本分析-python代码

C:\Users\asus\Desktop\数据中心\2019文本分析\电脑商品评论文本分析-python代码


 # 文本预处理

## 读取数据

In [3]:
data1 = pd.read_csv('联想-1000.csv',sep=',',header=0)
data2 = pd.read_csv('RedmiBook 14 -1000.csv',sep=',',header=0)
data3 = pd.read_csv('Apple 2019款 MacBook Pro 13.3-999.csv',sep=',',header=0)
data4 = pd.read_csv('荣耀MagicBook 14-547.csv',sep=',',header=0)
data_lst = [data1,data2,data3,data4]
data = pd.concat(data_lst,ignore_index=True)

## 查看数据

In [4]:
data

,Unnamed: 0,comment_id,product_id,guid,content,create_time,reference_id,reference_time,score,nickname,productColor,productSize,productId,referenceName,afterUserComment
0,0,13511270835,1.000030e+11,ddbeea7c-53ba-4ec3-9b12-982ed69cb729,老公给买的礼物，外观大气，开关机速度都很快，运行速度也很快，用着很满意，包装完好，物流给力,2019/11/23 12:51,1.000030e+11,2019/11/2 22:01,5,xiaoniu20021121,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
1,1,13511166583,1.000030e+11,dfe72168-fa9c-4deb-ac79-b2acf76bfdfe,运行速度：快\n屏幕效果：一般,2019/11/23 12:27,1.000030e+11,2019/11/5 10:37,5,j***D,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
2,2,13511078768,1.000030e+11,c4094e3c-2d62-4853-a6e6-7e1585d174d0,运行速度：快非常不错,2019/11/23 12:06,1.000030e+11,2019/11/18 15:06,5,j***d,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
3,3,13511049690,1.000030e+11,62192f36-d318-4af3-b04e-df81c0dfdc8e,京东服务很好，多次在京东买手机 电脑，都行。这次也没让我失望。这款电脑，配置强悍，虽然是第八...,2019/11/23 11:59,6.076609e+06,2019/11/20 21:12,5,j***1,翼480,Intel i7处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
4,4,13511024910,1.000030e+11,842120a5-6785-4351-b735-becafbd2fb0d,运行速度还可以，散热一般，刚买来开机挺快10s，软件安装多了现在就变成23s了，屏幕显示效果...,2019/11/23 11:54,1.000030e+11,2019/11/20 10:17,4,逝***7,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
5,5,13511002060,1.000030e+11,9f5e6397-e44c-4e4d-81ac-af66f70fcee1,运行速度：好卡\n其他特色：噪音大\n轻薄程度：一般,2019/11/23 11:49,1.000030e+11,2019/10/10 16:29,1,jd_189382zqo,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
6,6,13510804945,1.000030e+11,2fc982c2-37ae-472a-8187-b433f6d27d73,运行速度：运行流畅，开机10秒\n轻薄程度：电脑轻，薄\n外形外观：总体还可以\n公司购买,2019/11/23 11:07,1.000020e+11,2019/11/11 23:33,5,l***g,翼490,Intel i7处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
7,7,13510786584,1.000030e+11,b3493a6e-ff58-48a8-a484-a357b78e9895,刚买不到一月开机就开不了了，送到维修站点，说是里面一根线接触不良，要换新线，作为消费者，我觉...,2019/11/23 11:03,6.076609e+06,2019/10/21 21:40,5,z***0,翼480,Intel i7处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,这次购物让我很失望，京东老客户了，很是失望
8,8,13510712077,1.000030e+11,01e9430f-d9c5-4799-b044-f5521e86981d,我都不知道该对京东失望还是对联想失望！唯一一次再网上买电子产品，却两次产品问题！第一次内存条...,2019/11/23 10:47,6.072614e+06,2019/11/17 12:12,1,M***8,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN
9,9,13510644214,1.000030e+11,8a09a77e-b082-40c9-a9d1-255ee93110af,第一次买到这种内置电池的电脑。也很轻薄，很轻，很喜欢！,2019/11/23 10:33,6.072614e+06,2019/9/27 12:46,5,婕***园,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN


In [5]:
# 查看数据信息，注意到afterUserComment存在空值
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3546 entries, 0 to 3545
Data columns (total 15 columns):
Unnamed: 0          3546 non-null int64
comment_id          3546 non-null int64
product_id          3546 non-null float64
guid                3546 non-null object
content             3546 non-null object
create_time         3546 non-null object
reference_id        3546 non-null float64
reference_time      3546 non-null object
score               3546 non-null int64
nickname            3546 non-null object
productColor        3545 non-null object
productSize         2428 non-null object
productId           3546 non-null float64
referenceName       3546 non-null object
afterUserComment    207 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 415.6+ KB


## 增加一列总评 ‘totalcontent’

In [6]:
data['totalcontent']=data['content'].str.cat(data['afterUserComment'].apply(str))

In [7]:
data.head()

,Unnamed: 0,comment_id,product_id,guid,content,create_time,reference_id,reference_time,score,nickname,productColor,productSize,productId,referenceName,afterUserComment,totalcontent
0,0,13511270835,1.000030e+11,ddbeea7c-53ba-4ec3-9b12-982ed69cb729,老公给买的礼物，外观大气，开关机速度都很快，运行速度也很快，用着很满意，包装完好，物流给力,2019/11/23 12:51,1.000030e+11,2019/11/2 22:01,5,xiaoniu20021121,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN,老公给买的礼物，外观大气，开关机速度都很快，运行速度也很快，用着很满意，包装完好，物流给力nan
1,1,13511166583,1.000030e+11,dfe72168-fa9c-4deb-ac79-b2acf76bfdfe,运行速度：快\n屏幕效果：一般,2019/11/23 12:27,1.000030e+11,2019/11/5 10:37,5,j***D,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN,运行速度：快\n屏幕效果：一般nan
2,2,13511078768,1.000030e+11,c4094e3c-2d62-4853-a6e6-7e1585d174d0,运行速度：快非常不错,2019/11/23 12:06,1.000030e+11,2019/11/18 15:06,5,j***d,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN,运行速度：快非常不错nan
3,3,13511049690,1.000030e+11,62192f36-d318-4af3-b04e-df81c0dfdc8e,京东服务很好，多次在京东买手机 电脑，都行。这次也没让我失望。这款电脑，配置强悍，虽然是第八...,2019/11/23 11:59,6.076609e+06,2019/11/20 21:12,5,j***1,翼480,Intel i7处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN,京东服务很好，多次在京东买手机 电脑，都行。这次也没让我失望。这款电脑，配置强悍，虽然是第八...
4,4,13511024910,1.000030e+11,842120a5-6785-4351-b735-becafbd2fb0d,运行速度还可以，散热一般，刚买来开机挺快10s，软件安装多了现在就变成23s了，屏幕显示效果...,2019/11/23 11:54,1.000030e+11,2019/11/20 10:17,4,逝***7,翼480,Intel i5处理器,1.000030e+11,联想ThinkPad 翼480（4VCD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-82...,NaN,运行速度还可以，散热一般，刚买来开机挺快10s，软件安装多了现在就变成23s了，屏幕显示效果...


## 去除标点

In [8]:
#去除标点函数
def punc(string):
    # import re
    # from string import punctuation #标点符号
    punc = punctuation+u'《》？，！“”#.,;《》？！“”‘’@￥%…&×（）——+【】{};；●。&～、|\s:：'  #u可以把引号里面的字符转成unicode编码
    string=re.sub("[{}]".format(punc), " ", string)  # 自定义搜索规则[]：其中任一字符；替换成“ ”；搜索文本是string
    return string

In [9]:
#去除标点
data['totalcontent']=data['totalcontent'].apply(punc)
data['totalcontent']

0        老公给买的礼物 外观大气 开关机速度都很快 运行速度也很快 用着很满意 包装完好 物流给力nan
1                                       运行速度 快 屏幕效果 一般nan
2                                           运行速度 快非常不错nan
3       京东服务很好 多次在京东买手机 电脑 都行 这次也没让我失望 这款电脑 配置强悍 虽然是第八...
4       运行速度还可以 散热一般 刚买来开机挺快10s 软件安装多了现在就变成23s了 屏幕显示效果...
5                             运行速度 好卡 其他特色 噪音大 轻薄程度 一般nan
6           运行速度 运行流畅 开机10秒 轻薄程度 电脑轻 薄 外形外观 总体还可以 公司购买nan
7       刚买不到一月开机就开不了了 送到维修站点 说是里面一根线接触不良 要换新线 作为消费者 我觉...
8       我都不知道该对京东失望还是对联想失望 唯一一次再网上买电子产品 却两次产品问题 第一次内存条...
9                          第一次买到这种内置电池的电脑 也很轻薄 很轻 很喜欢 nan
10                                 感觉还行 运行正常 办公完全没啥问题 nan
11                 运行速度 很快 屏幕效果 一般 散热性能 不容易发热 外形外观 很轻薄nan
12      运行速度 快 屏幕效果 很清楚 散热性能 比较好 外形外观 很有质感 轻薄程度 是轻薄本 看...
13                                         速度还可以 系统稳定 nan
14                         很流畅 速度快 散热好几乎听不到声音 京东自营值得信赖nan
15      屏幕效果 还不错 运行速度 很快奥 外形外观 还可以是我想要的 其他特色 送货速度快快递员小...
16            老公用了几天 感觉运行速度比之前的快多了 屏幕画面也清晰了许多 轻薄更方便携带了nan
17      运行速度 很

## 分词+去停用词

In [12]:
#导入停用词表
#import codecs
stop_words = '停用词表.txt'
stopwords = codecs.open(stop_words,'r',encoding='utf8').readlines()  #unicode文件读写，'r'代表读
stopwords = [ w.strip() for w in stopwords ]  # 列表推导式；strip()删除字符串两端空白符
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']  # 停用词词性
# stopwords

In [13]:
#分词结果导入txt
#from pandas.core.frame import DataFrame
#import jieba.posseg as pseg
fenciurl='分词结果1.txt'#分词文档链接
fh = open(fenciurl, 'w', encoding='utf-8')
# 写文件
for i in data['totalcontent']:
    words=pseg.cut(i) #分词：generator对象，包括词语和词性jieba
#     print(list(words))
    for word,flag in words:
        #if word not in stopwords and flag not in stop_flag:
        if word not in (['nan','n'] + stopwords) and flag not in stop_flag:
            fh.write(word+" ")
#            print(flag)#词性 https://blog.csdn.net/Yellow_python/article/details/83991967
    fh.write("\n")
fh.close()

# 文本表示

In [14]:
#分词结果导入
text = []
with open(fenciurl,encoding='UTF-8') as f:
    for line in f:
        text.append(line.strip().split()) # split()分个字符串为列表，默认为所有的空字符，包括空格、换行(\n)、制表符(\t)等
print(len(text))
print(text)

3546
[['老公', '给买', '礼物', '外观', '大气', '开关机', '速度', '运行', '速度', '满意', '包装', '完好', '物流', '力'], ['运行', '速度', '屏幕', '效果'], ['运行', '速度', '不错'], ['京东', '服务', '京东', '买手机', '电脑', '都行', '失望', '电脑', '配置', '强悍', '内存', '存', '空间', '很大', '缓存', '独显', '组合', '散热', '反应速度快', '学习', '办公', '杠杠'], ['运行', '速度', '散热', '买来', '开机', '挺快', 's', '软件', '安装', 's', '屏幕显示', '效果', '开', '小卡', '倒', '还好'], ['运行', '速度', '卡', '特色', '噪音', '轻薄', '程度'], ['运行', '速度', '运行', '流畅', '开机', '轻薄', '程度', '电脑', '轻', '薄', '外形', '外观', '总体', '公司', '购买'], ['买不到', '开机', '开', '送到', '维修', '站点', '说', '线', '接触不良', '要换', '新线', '消费者', '联想', '品质', '出厂', '检验', '保障', '耽搁', '工作', '希望', '联想公司', '说法', '购物', '失望', '京东', '老客户', '失望'], ['京东', '失望', '联想', '失望', '唯一', '网上', '买', '电子产品', '产品', '内存条', '故障', '退换货', '折腾', '麻烦', '不到', '屏幕', '闪屏', '服', '图', '图', '心', '塞', '差', '评差', '评', '网上', '买', '东西'], ['买', '内置', '电池', '电脑', '轻薄', '轻', '喜欢'], ['感觉', '行', '运行', '办公'], ['运行', '速度', '屏幕', '效果', '散热', '性能', '发热', '外形', '外观', '轻薄'], ['运行', '速度', '屏幕', '效果', '散热', '性能

## 词袋模型&TF-IDF

In [15]:
#from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(text) #提取所有不同的词语
doc_vectors = [dictionary.doc2bow(t) for t in text]#词袋模型 词语索引+词频
print(len(doc_vectors)) 
print(len(dictionary))
print(doc_vectors)

3546
5519
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2)], [(11, 1), (12, 1), (13, 1), (14, 1)], [(11, 1), (12, 1), (15, 1)], [(16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)], [(11, 1), (12, 1), (14, 1), (26, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)], [(11, 1), (12, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)], [(2, 1), (11, 2), (12, 1), (30, 1), (43, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)], [(17, 1), (21, 2), (42, 1), (43, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1)], [(13, 1), (17, 1), (21, 2), (74, 1), (81, 1), (82, 1), (83, 2), (8

>TF-IDF模型

TF表示的是词频：即这个词在一篇文档中出现的频率

idf表示的是逆文档频率， 即log(文档的个数/1+出现该词的文档个数)  可以看出出现该词的文档个数越小，表示这个词越稀有，在这篇文档中也是越重要的

TF-idf： 表示TF*idf, 即词频*逆文档频率

词袋模型不仅考虑了一个词的词频，同时考虑了这个词在整个语料库中的重要性

In [16]:
#TF-IDF模型
tfidf = models.TfidfModel(doc_vectors)
tfidf_vectors = tfidf[doc_vectors]#词袋-tfidf向量
print(len(tfidf_vectors))
print(tfidf_vectors[0])
print(tfidf_vectors[1])
#index = similarities.MatrixSimilarity(tfidf_vectors)

3546
[(0, 0.26434107968629694), (1, 0.23741973159543325), (2, 0.0679517308542594), (3, 0.23741973159543325), (4, 0.3654033544671313), (5, 0.34781402681731194), (6, 0.17428799184912777), (7, 0.19177798308486352), (8, 0.39019411941227505), (9, 0.4573649769523818), (10, 0.3383890154086552), (11, 0.05787900382346988), (12, 0.10876356263695335)]
[(11, 0.4463693758684398), (12, 0.4193984032582672), (13, 0.5313810147721668), (14, 0.585229507775315)]


## word2vec模型

> 其实，要得到好的word2vec词向量，需要用大的语料数据（100G+）来训练，这里为了教学展示只使用了爬取的评论来作为训练语料数据。
具体中文语料数据下载，可通过下方链接
* 搜狗实验室https://www.sogou.com/labs/resource/ca.php

![title](word2vec参数.png)

In [17]:
#训练并存储word2vec模型
#import logging
#from gensim.models.word2vec import LineSentence, Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)#加载日志
#日志记录时的文本时间"%(asctime)s"；打印日志级别的名称%(levelname)s；打印当前执行程序名%(filename)s
sentences= LineSentence(fenciurl) #按行读取文件中的每一行
model = Word2Vec(sentences,window=5, min_count=1, iter=1000)
#window:表示当前词与预测词在一个句子中的最大距离,min_count: 可以对字典做截断. 词频少于min_count次数的单词会被丢弃掉, 默认值为5
#默认CBOW算法，window窗口大小
modelurl='w2vmod.mod'#Word2Vec模型存储路径
model.save(modelurl)

2020-08-15 17:56:54,516 : INFO : collecting all words and their counts
2020-08-15 17:56:54,519 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-15 17:56:54,547 : INFO : collected 5519 word types from a corpus of 52931 raw words and 3506 sentences
2020-08-15 17:56:54,548 : INFO : Loading a fresh vocabulary
2020-08-15 17:56:54,561 : INFO : effective_min_count=1 retains 5519 unique words (100% of original 5519, drops 0)
2020-08-15 17:56:54,562 : INFO : effective_min_count=1 leaves 52931 word corpus (100% of original 52931, drops 0)
2020-08-15 17:56:54,586 : INFO : deleting the raw counts dictionary of 5519 items
2020-08-15 17:56:54,590 : INFO : sample=0.001 downsamples 43 most-common words
2020-08-15 17:56:54,591 : INFO : downsampling leaves estimated 39164 word corpus (74.0% of prior 52931)
2020-08-15 17:56:54,634 : INFO : estimated required memory for 5519 words and 100 dimensions: 7174700 bytes
2020-08-15 17:56:54,637 : INFO : resetting layer weights
2

2020-08-15 17:56:56,138 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:56,141 : INFO : EPOCH - 18 : training on 52931 raw words (39177 effective words) took 0.1s, 514574 effective words/s
2020-08-15 17:56:56,199 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:56,201 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:56:56,207 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:56,208 : INFO : EPOCH - 19 : training on 52931 raw words (39194 effective words) took 0.1s, 611316 effective words/s
2020-08-15 17:56:56,259 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:56,260 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:56:56,266 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:56,269 : INFO : EPOCH - 20 : training on 52931 raw words (39135 effective word

2020-08-15 17:56:57,434 : INFO : EPOCH - 38 : training on 52931 raw words (39183 effective words) took 0.1s, 709004 effective words/s
2020-08-15 17:56:57,490 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:57,494 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:56:57,505 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:57,506 : INFO : EPOCH - 39 : training on 52931 raw words (39024 effective words) took 0.1s, 562680 effective words/s
2020-08-15 17:56:57,579 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:57,587 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:56:57,594 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:57,595 : INFO : EPOCH - 40 : training on 52931 raw words (39212 effective words) took 0.1s, 455250 effective words/s
2020-08-15 17:56:57,661 : INFO : worker thread finis

2020-08-15 17:56:58,814 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:58,815 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:56:58,822 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:58,824 : INFO : EPOCH - 59 : training on 52931 raw words (39148 effective words) took 0.1s, 679715 effective words/s
2020-08-15 17:56:58,873 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:58,875 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:56:58,884 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:56:58,885 : INFO : EPOCH - 60 : training on 52931 raw words (39044 effective words) took 0.1s, 662733 effective words/s
2020-08-15 17:56:58,936 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:56:58,937 : INFO : worker thread finished; awaiting finish of 1 more threads
2020

2020-08-15 17:57:00,104 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:00,111 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:00,112 : INFO : EPOCH - 79 : training on 52931 raw words (39169 effective words) took 0.1s, 711367 effective words/s
2020-08-15 17:57:00,162 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:00,164 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:00,171 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:00,172 : INFO : EPOCH - 80 : training on 52931 raw words (39033 effective words) took 0.1s, 679213 effective words/s
2020-08-15 17:57:00,225 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:00,228 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:00,237 : INFO : worker thread finished; awaiting finish of 0 more threads
2020

...

2020-08-15 17:57:49,425 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:49,427 : INFO : EPOCH - 909 : training on 52931 raw words (39113 effective words) took 0.1s, 700913 effective words/s
2020-08-15 17:57:49,477 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:49,480 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:49,489 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:49,490 : INFO : EPOCH - 910 : training on 52931 raw words (39158 effective words) took 0.1s, 644199 effective words/s
2020-08-15 17:57:49,546 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:49,549 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:49,556 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:49,557 : INFO : EPOCH - 911 : training on 52931 raw words (39223 effective w

2020-08-15 17:57:50,632 : INFO : EPOCH - 929 : training on 52931 raw words (39061 effective words) took 0.1s, 683295 effective words/s
2020-08-15 17:57:50,676 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:50,679 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:50,685 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:50,687 : INFO : EPOCH - 930 : training on 52931 raw words (39065 effective words) took 0.1s, 730452 effective words/s
2020-08-15 17:57:50,738 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:50,739 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:50,745 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:50,746 : INFO : EPOCH - 931 : training on 52931 raw words (39069 effective words) took 0.1s, 688269 effective words/s
2020-08-15 17:57:50,792 : INFO : worker thread fi

2020-08-15 17:57:51,937 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:51,939 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:51,946 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:51,947 : INFO : EPOCH - 950 : training on 52931 raw words (39102 effective words) took 0.1s, 666092 effective words/s
2020-08-15 17:57:51,996 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:51,999 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:52,005 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:52,007 : INFO : EPOCH - 951 : training on 52931 raw words (39111 effective words) took 0.1s, 676190 effective words/s
2020-08-15 17:57:52,062 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:52,066 : INFO : worker thread finished; awaiting finish of 1 more threads
20

2020-08-15 17:57:53,165 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:53,168 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:53,170 : INFO : EPOCH - 970 : training on 52931 raw words (39186 effective words) took 0.1s, 722060 effective words/s
2020-08-15 17:57:53,217 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:53,219 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:53,225 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:53,226 : INFO : EPOCH - 971 : training on 52931 raw words (39090 effective words) took 0.1s, 716176 effective words/s
2020-08-15 17:57:53,275 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:53,278 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:53,285 : INFO : worker thread finished; awaiting finish of 0 more threads
20

2020-08-15 17:57:54,358 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:54,359 : INFO : EPOCH - 990 : training on 52931 raw words (39222 effective words) took 0.1s, 674866 effective words/s
2020-08-15 17:57:54,409 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:54,413 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:54,420 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:54,421 : INFO : EPOCH - 991 : training on 52931 raw words (39188 effective words) took 0.1s, 661644 effective words/s
2020-08-15 17:57:54,474 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-15 17:57:54,476 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-15 17:57:54,478 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-15 17:57:54,479 : INFO : EPOCH - 992 : training on 52931 raw words (39120 effective w

In [18]:
#加载w2v模型
#import numpy as np
model_loaded = Word2Vec.load(modelurl)
#词向量的维度size=100,上下文窗口window=5,负采样数negative=5

#形成文本特征矩阵feature
f0=model_loaded[text[0]]#文本向量矩阵
feature=np.array([f0.sum(axis=0)/f0.shape[0]])#取平均作为特征
print(len(f0))
print(f0)

for a,i in enumerate(text):
    if a!=0 and len(i)!=0:
        f0=model_loaded[i]
        f1=f0.sum(axis=0)/f0.shape[0]
        feature=np.insert(feature,feature.shape[0],values=f1,axis=0)
    elif a!=0 and len(i)==0:
        feature=np.insert(feature,feature.shape[0],values=np.zeros(100),axis=0)

2020-08-15 17:58:40,943 : INFO : loading Word2Vec object from w2vmod.mod
2020-08-15 17:58:41,061 : INFO : loading wv recursively from w2vmod.mod.wv.* with mmap=None
2020-08-15 17:58:41,061 : INFO : setting ignored attribute vectors_norm to None
2020-08-15 17:58:41,061 : INFO : loading vocabulary recursively from w2vmod.mod.vocabulary.* with mmap=None
2020-08-15 17:58:41,061 : INFO : loading trainables recursively from w2vmod.mod.trainables.* with mmap=None
2020-08-15 17:58:41,061 : INFO : setting ignored attribute cum_table to None
2020-08-15 17:58:41,061 : INFO : loaded w2vmod.mod
C:\python\anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
C:\python\anaconda\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


14
[[-1.0508388   0.34685028 -2.6140432  ... -1.5081711   0.03477826
  -0.6623042 ]
 [-1.5291501  -0.8629724  -0.38454986 ... -0.45337662  0.7434315
   0.7006433 ]
 [-4.8950257  -0.55870044  0.23145656 ... -0.57821715  2.6585653
  -0.478305  ]
 ...
 [-3.2260811  -3.3710191   0.7994294  ...  0.5381297   0.3270629
   0.68831754]
 [-5.9261794   0.28867793  0.12963708 ...  1.6989553   0.10214663
  -1.8236773 ]
 [ 1.7816648   2.3690429   0.24479178 ...  1.1948677  -4.074606
  -1.1207817 ]]


In [19]:
print(len(feature))
print(len(feature[0]))
print(feature[0])

3546
100
[-1.1598322  -0.876583    0.31839964  0.35827324 -1.4376963   0.98729414
 -1.3295939   0.06181436  0.53361374  1.0872642   1.1497695   0.02372677
 -1.4164876  -1.4012063  -1.0492908   0.2823804   0.4167547   0.48193285
 -0.28168556 -1.621728   -0.75216115  0.47171196  1.7230732  -1.5195749
  0.62453383  0.28680405 -0.61383516 -0.5188268  -0.08992313  0.97578305
  2.1863093   0.05393478 -0.82815176 -1.3277823  -1.6033405  -2.1645055
 -0.5629779   0.81477726 -0.04147651  1.3107252  -0.5893261   0.75596243
  1.5135466  -0.5652519  -0.03389022  0.42142084 -0.7044283  -0.15593982
  0.09735759 -0.2515758  -0.1413082  -0.6535906   1.3126986   0.22406735
 -0.3078196  -1.1690568  -0.34677497 -1.7570684  -0.74065405  0.8713277
  0.8226604   0.75588894 -0.8834977  -0.20499183  1.2532208   0.79735285
 -0.7701395   0.41406956 -0.5852662  -0.13983057  0.56254584  1.8359597
  1.0240886   1.198663   -0.2799063  -0.08965387  0.72953427  0.3130913
 -0.12484096 -0.8065465  -0.46068683 -1.1717771

# 基于情感词典方法的情感分析

In [20]:
import pandas as pd

In [21]:
# 加载情感词典（Boson情感词典）
df= pd.read_table("BosonNLP_sentiment_score.txt",sep= " ",names=['key','score'])

C:\python\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [22]:
key = df['key'].values.tolist()
score = df['score'].values.tolist()
# 导入经过预处理的分词结果
df_comments = pd.read_table("分词结果.txt",encoding="utf-8",header=None)
df_comments.columns=["content"]
df_comments

C:\python\anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


,content
0,老公 给买 礼物 外观 大气 开关机 速度 运行 速度 着 满意 包装 完好 物流 力
1,运行 速度 快 屏幕 效果 一般
2,运行 速度 快 不错
3,京东 服务 好 京东 买手机 电脑 都行 让 失望 电脑 配置 强悍 是 内存 大 存 空间...
4,运行 速度 散热 一般 买来 开机 挺快 s 软件 安装 了 变成 s 了 屏幕显示 效果 ...
5,运行 速度 好 卡 特色 噪音 大 轻薄 程度 一般
6,运行 速度 运行 流畅 开机 轻薄 程度 电脑 轻 薄 外形 外观 总体 公司 购买
7,买不到 开机 开 不了 了 送到 维修 站点 说 是 线 接触不良 要换 新线 作为 消费者...
8,知道 京东 失望 联想 失望 唯一 网上 买 电子产品 产品 问题 内存条 故障 退换货 折...
9,买 到 内置 电池 电脑 轻薄 很 轻 很 喜欢


In [23]:
# 计算得分函数
def sen_scores(words):
    sum_score = 0
    for word in words:
        if word in key:
            word_score = score[key.index(word)]
            sum_score = word_score + sum_score
    return sum_score

# 合并得分
def scores(comments):
    words=[]
    score_list=[]
    count=0
    for comment in comments:   
        words = comment.split(" ")
        score_list.append(round(sen_scores(words),3))
        
        # 打印动态进度条
        count = count+1
        charm_num = count // int(len(df_comments)/100)
        if charm_num in range(1,101):
            print("\r"+"进度：{0}{1}%".format('*'*charm_num ,charm_num), end='')
    
    df_comments["scores"]=score_list
    return df_comments

In [24]:
# 计算得分(需要几分钟的时间)
scores(df_comments["content"]) 

进度：****************************************************************************************************100%

,content,scores
0,老公 给买 礼物 外观 大气 开关机 速度 运行 速度 着 满意 包装 完好 物流 力,15.314
1,运行 速度 快 屏幕 效果 一般,2.569
2,运行 速度 快 不错,3.953
3,京东 服务 好 京东 买手机 电脑 都行 让 失望 电脑 配置 强悍 是 内存 大 存 空间...,16.950
4,运行 速度 散热 一般 买来 开机 挺快 s 软件 安装 了 变成 s 了 屏幕显示 效果 ...,10.607
5,运行 速度 好 卡 特色 噪音 大 轻薄 程度 一般,5.834
6,运行 速度 运行 流畅 开机 轻薄 程度 电脑 轻 薄 外形 外观 总体 公司 购买,11.148
7,买不到 开机 开 不了 了 送到 维修 站点 说 是 线 接触不良 要换 新线 作为 消费者...,4.611
8,知道 京东 失望 联想 失望 唯一 网上 买 电子产品 产品 问题 内存条 故障 退换货 折...,-3.550
9,买 到 内置 电池 电脑 轻薄 很 轻 很 喜欢,4.166


In [25]:
# 图形展示——得分图
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(15,10))

def plotscore(a,b,name_string):
    x = np.arange(b-a)
    y = df_comments["scores"][a:b].sort_values()
    plt.bar(x, y,color='royalblue')
    plt.ylabel("Score")
    plt.xlabel("Comments num")
    plt.title(name_string+"'s Comments Score Distribution")
    plt.axhline(y=20, color="black", linestyle=":",lw=1.5)
    plt.axhline(y=20, color="black", linestyle=":",lw=1.5)

# 联想 0-999
plt.subplot(2,2,1)
plotscore(0,999,"Lenovo")
# 小米 1000-1999
plt.subplot(2,2,2)
plotscore(1000,1999,"Xiaomi")
# 苹果 2000-2998
plt.subplot(2,2,3)
plotscore(2000,2998,"Apple")
# 华为 2999-3539
plt.subplot(2,2,4)
plotscore(2999,3539,"Huawei")

In [26]:
# 打印分数最高和最低两组评论
def print_comment(category,num):
    for i in range(num):
        if category == "low":
            category_low = df_comments["scores"].sort_values()[:num]  
            l = category_low.index[i]
            print("low"+"["+str(i+1)+"]:"+str(df_comments["scores"][l])+" "+df_comments["content"][l].replace(" ","")+"\n")
        else:
            category_high = df_comments["scores"].sort_values()[-num:]  
            h = category_high.index[i]
            print("high"+"["+str(i+1)+"]:"+str(df_comments["scores"][h])+" "+df_comments["content"][h].replace(" ","")+"\n")

print_comment("low",2)
print_comment("high",2)

low[1]:-25.307 再也不会来京东买东西了充满电看电视剧小时编辑视频小时连续死机只能强制重启电脑机身热烫手没事风扇响声音大嗡嗡嗡垃圾垃圾垃圾客服垃圾退差价扣除赠品钱垃圾垃圾垃圾垃圾

low[2]:-24.979 下单签收发现了故障问题电池用到了会突然关机也就是说电池容量决定退回去换货说是笔记本检修天无力吐槽京东失望apple售后失望明显大问题小时能检测到问题直接换新不就行了吗需要监测欺负消费者了吧有消费者考虑吗星不想辣鸡京东买macbookpro收到货后发现电池用到关机黑屏发现质量问题换货磨磨蹭蹭了说重装了系统修好了硬件方面没问题不予换货到处理好坑惨了

high[1]:119.368 运行速度很流畅屏幕效果色域能轻薄程度不错哦外形外观适合质量好卖家描述完全一致满意很喜欢收到了包装会买用着不错产品相满意好很不错搞活动买到不错习惯了觉得着包装完整使用过评价效果发货速度快包装仔细严实物流公司服务态度好运送速度很满意购物不错正品活动优惠性价比超高送东西好买会买用着不错产品满意好很不错效果好会买不错用起来好设计合理值得拥有宝贝早收到了好喜欢活动买超好超实惠谢谢啦好评

high[2]:126.8 运行速度运行速度行新机固态不行知道评论了屏幕效果屏幕是一大亮点显示很舒服色域有特色有特色而已性价比高肉眼分辨明显拿设备对比会感觉出来显得艳丽散热性能笔记本吐糟点金属散热烫手煎鸡蛋敢说烫告诉原因吧开机烫安装软件煎鸡蛋是散热性铜管打风扇风量不行外形外观外观是唯一比较满意全金属摸着看着说服很爽轻薄程度厚重起来没什么特点唯一特点金属牌子超薄金属呢特色买本子请注意选择不要支持国产爱国买华为支持国产支持华为品牌感觉后悔买华为说购买体验吧官方宣传页面上期分期免息进去免息购买降价京东好帮解决了这两点不错笔记本天到是买店看中京东物流特地京东购买买来是国产系统软件小游戏无法使用只能装微软送了u盘要笔记本收货有维修请看追评华为要小加装固态不能自己动手动手华为官方解释影响售后意思加无售后了联系当地华为维修点站点说没有华为笔记本拆机权限无法帮忙加装固态维修点是华为客服介绍只能邮寄到外地加装固态邮寄费用是免了特地官方电话客服咨询加装是否收费还好有聊天记录拆机费用黑黑hellip加装不了华为加装非要收费邮寄问了需要提供固态吗说是没有邮寄电源线固态固定支架收到说没有固态支架电源线无法加装要邮寄回来定配